In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import make_scorer, accuracy_score
import xgboost as xgb

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import initializers

from matplotlib import pyplot
from sklearn.metrics import make_scorer, accuracy_score,mean_squared_error
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
#LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

2024-01-30 11:15:00.250991: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 11:15:00.251069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 11:15:00.316398: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 11:15:00.445653: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-30 11:15:01.606717: W tensorflow/compiler/tf2

# One-h-k

In [ ]:
def one_hot_encode(dataset, variables):
    for variable in variables:
        dummies = pd.get_dummies(dataset[variable], prefix=variable).astype(int)
        dataset = pd.concat([dataset, dummies], axis=1)
        dataset.drop(columns=[variable], inplace=True)
    return dataset

# Function Search

# MONK 1

In [ ]:
df = pd.read_csv('MONK/monks-1.train', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])
df_2 = pd.read_csv('MONK/monks-1.test', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])


## Applicazione on hot encoding

In [ ]:
monk1_tr=one_hot_encode(df, [1,2,3,4,5,6])
monk1_ts=one_hot_encode(df_2, [1,2,3,4,5,6])

## Split dataset

In [ ]:
training_1_x, validation_1_x, training_1_y, validation_1_y = train_test_split(monk1_tr, df[['y']], stratify =None, test_size=0.30, random_state = 42, shuffle=False)

In [ ]:
x_1_ts = monk1_ts.iloc[:,1:].values #ottiene tutte le colonne con indice >=1, scartando la target label
y_1_ts = monk1_ts.iloc[:,0].values # ottiene solo colonne con indice 0, solo la target label
x_1_tr = training_1_x.iloc[:,1:].values
y_1_tr = training_1_y.iloc[:,:].values
x_1_vl = validation_1_x.iloc[:,1:].values
y_1_vl = validation_1_y.iloc[:,:].values

x_1_ds = monk1_tr.iloc[:,1:].values
y_1_ds = monk1_tr.iloc[:,0].values

## Search

valutiamo il modello rispetto al numero di round di boosting.

CPU times: user 1min 54s <br>
Sys: 17.6 s <br>
Total: 2min 12s <br>
Wall time: 2min 12s

In [ ]:
%%time


# Define the parameter grid
lambda_values = [0,1,2]
max_depth_values = [3, 6, 9]
learning_rate_values = [0.05, 0.1, 0.20]
min_child_weight_values = [1, 10, 100]

# Prepare to collect results
best_params = None
best_score = float("inf")  # Initialize with a large number
best_num_boost_round = None
combination_count = 0  # Counter for combinations

# Calculate the total number of combinations
total_combinations = len(lambda_values) * len(max_depth_values) * len(learning_rate_values) * len(min_child_weight_values)
print("Total combinations: ", total_combinations)
xgb_train_matrix = xgb.DMatrix(x_1_ds,label=y_1_ds)
# Loop over the hyperparameter values
for lambda_value in lambda_values:
    for max_depth in max_depth_values:
        for learning_rate in learning_rate_values:
            for min_child_weight in min_child_weight_values:
                combination_count += 1  # Increment combination count
                params = {
                    "objective": "binary:logistic",
                    "max_depth": max_depth,
                    "learning_rate": learning_rate,
                    "subsample": 0.8,  # Assuming a default subsample value
                    "gamma": 0,  # Assuming a default gamma value
                    "booster": 'gbtree',
                    "min_child_weight": min_child_weight,
                    "lambda": lambda_value,  # Set the L2 regularization strength
                    "device": "cuda"
                }

                # Perform cross-validation with early stopping
                cv_results = xgb.cv(
                    params, xgb_train_matrix,
                    num_boost_round=1000,
                    nfold=5,
                    metrics=["rmse"],  # Assuming you're looking for RMSE as a regression metric
                    seed=42, early_stopping_rounds=10
                )

                # Compute average RMSE across folds
                avg_rmse = cv_results['test-rmse-mean'].min() * cv_results['test-rmse-mean'].min()
                optimal_rounds = cv_results.shape[0]
                if avg_rmse < best_score:
                    best_score = avg_rmse
                    best_params = params
                    best_num_boost_round = optimal_rounds  # Updated to optimal rounds

                # Print information about the current combination
                print(f"Combination {combination_count}/{total_combinations} - lambda: {lambda_value}, max_depth: {max_depth}, learning_rate: {learning_rate}, min_child_weight: {min_child_weight}, MSE: {avg_rmse}, Optimal Rounds: {optimal_rounds}")

print("Best Parameters:", best_params)
print("Optimal num_boost_round:", best_num_boost_round)
print("Best Score (MSE):", best_score)
print("Total Combinations Processed:", total_combinations)


Total combinations:  81
Combination 1/81 - lambda: 0, max_depth: 3, learning_rate: 0.05, min_child_weight: 1, MSE: 0.027620087394803695, Optimal Rounds: 303
Combination 2/81 - lambda: 0, max_depth: 3, learning_rate: 0.05, min_child_weight: 10, MSE: 0.2539678439642546, Optimal Rounds: 3
Combination 3/81 - lambda: 0, max_depth: 3, learning_rate: 0.05, min_child_weight: 100, MSE: 0.2540014588645318, Optimal Rounds: 1
Combination 4/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 1, MSE: 0.025240713688437118, Optimal Rounds: 121
Combination 5/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 10, MSE: 0.2539327833376049, Optimal Rounds: 3
Combination 6/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 100, MSE: 0.2540014588645318, Optimal Rounds: 1
Combination 7/81 - lambda: 0, max_depth: 3, learning_rate: 0.2, min_child_weight: 1, MSE: 0.01831438424612282, Optimal Rounds: 100
Combination 8/81 - lambda: 0, max_depth: 3, learning_rate: 0

CPU times: user 617 ms <br>
Sys: 11.2 ms <br>
Total: 628 ms <br>
Wall time: 368 ms

In [ ]:
%%time
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
xgb_train_matrix = xgb.DMatrix(x_1_ds,label=y_1_ds)
params ={'objective': 'binary:logistic', 'max_depth': 3, 'learning_rate': 0.2, 'subsample': 0.8, 'gamma': 0, 'booster': 'gbtree', 'min_child_weight': 1, 'lambda': 0, 'device': 'cuda'}
# Assuming 'dtrain' is already defined
model = xgb.train(params, xgb_train_matrix, num_boost_round=100, evals=[(xgb_train_matrix, "train")],
                  early_stopping_rounds=100, verbose_eval=False)

dtest = xgb.DMatrix(x_1_ts, label=y_1_ts)

# Predict labels for the training set and test set, converting probabilities to binary output
train_labels_pred = (model.predict(xgb_train_matrix) > 0.5).astype(int)
test_labels_pred = (model.predict(dtest) > 0.5).astype(int)

# Calculate accuracy for training and test datasets
train_accuracy = accuracy_score(y_1_ds, train_labels_pred)
test_accuracy = accuracy_score(y_1_ts, test_labels_pred)

# Output the results
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

train_mse = mean_squared_error(y_1_ds, train_labels_pred)
test_mse = mean_squared_error(y_1_ts, test_labels_pred)
print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")


Training Accuracy: 1.0
Test Accuracy: 1.0
Training MSE: 0.0
Test MSE: 0.0
CPU times: user 617 ms, sys: 11.2 ms, total: 628 ms
Wall time: 368 ms


# MONK 2

In [ ]:
df = pd.read_csv('MONK/monks-2.train', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])
df_2 = pd.read_csv('MONK/monks-2.test', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])


## Applicazione on hot encoding e split dataset

In [ ]:
monk2_tr=one_hot_encode(df, [1,2,3,4,5,6])
monk2_ts=one_hot_encode(df_2, [1,2,3,4,5,6])
training_2_x, validation_2_x, training_2_y, validation_2_y = train_test_split(monk2_tr, df[['y']], stratify =None, test_size=0.30, random_state = 42, shuffle=False)

x_2_ts = monk2_ts.iloc[:,1:].values #ottiene tutte le colonne con indice >=1, scartando la target label
y_2_ts = monk2_ts.iloc[:,0].values # ottiene solo colonne con indice 0, solo la target label

x_2_tr = training_2_x.iloc[:,1:].values
y_2_tr = training_2_y.iloc[:,:].values
x_2_vl = validation_2_x.iloc[:,1:].values
y_2_vl = validation_2_y.iloc[:,:].values

x_2_ds = monk2_tr.iloc[:,1:].values
y_2_ds = monk2_tr.iloc[:,0].values


## Search

CPU times: user 2min 14s <br>
Sys: 22.5 s <br>
Total: 2min 36s <br>
Wall time: 2min 36s

In [ ]:
%%time


# Define the parameter grid
lambda_values = [0,1,2]
max_depth_values = [3, 6, 9]
learning_rate_values = [0.01, 0.1, 0.2]
min_child_weight_values = [1, 5, 10]

# Prepare to collect results
best_params = None
best_score = float("inf")  # Initialize with a large number
best_num_boost_round = None
combination_count = 0  # Counter for combinations

# Calculate the total number of combinations
total_combinations = len(lambda_values) * len(max_depth_values) * len(learning_rate_values) * len(min_child_weight_values)
print("Total combinations: ", total_combinations)
xgb_train_matrix = xgb.DMatrix(x_2_ds,label=y_2_ds)
# Loop over the hyperparameter values
for lambda_value in lambda_values:
    for max_depth in max_depth_values:
        for learning_rate in learning_rate_values:
            for min_child_weight in min_child_weight_values:
                combination_count += 1  # Increment combination count
                params = {
                    "objective": "binary:logistic",
                    "max_depth": max_depth,
                    "learning_rate": learning_rate,
                    "subsample": 0.8,  # Assuming a default subsample value
                    "gamma": 0,  # Assuming a default gamma value
                    "booster": 'gbtree',
                    "min_child_weight": min_child_weight,
                    "lambda": lambda_value,  # Set the L2 regularization strength
                    "device": "cuda"
                }

                # Perform cross-validation with early stopping
                cv_results = xgb.cv(
                    params, xgb_train_matrix,
                    num_boost_round=1000,
                    nfold=10,
                    metrics=["rmse"],
                    seed=42, early_stopping_rounds=50
                )

                # Compute average RMSE across folds
                avg_rmse = cv_results['test-rmse-mean'].min() * cv_results['test-rmse-mean'].min()
                optimal_rounds = cv_results.shape[0]
                if avg_rmse < best_score:
                    best_score = avg_rmse
                    best_params = params
                    best_num_boost_round = optimal_rounds

                # Print information about the current combination
                print(f"Combination {combination_count}/{total_combinations} - lambda: {lambda_value}, max_depth: {max_depth}, learning_rate: {learning_rate}, min_child_weight: {min_child_weight}, MSE: {avg_rmse}, Optimal Rounds: {optimal_rounds}")

print("Best Parameters:", best_params)
print("Optimal num_boost_round:", best_num_boost_round)
print("Best Score (MSE):", best_score)
print("Total Combinations Processed:", total_combinations)


Total combinations:  81
Combination 1/81 - lambda: 0, max_depth: 3, learning_rate: 0.01, min_child_weight: 1, MSE: 0.20121476480088504, Optimal Rounds: 682
Combination 2/81 - lambda: 0, max_depth: 3, learning_rate: 0.01, min_child_weight: 5, MSE: 0.21855311242019115, Optimal Rounds: 1000
Combination 3/81 - lambda: 0, max_depth: 3, learning_rate: 0.01, min_child_weight: 10, MSE: 0.2395112796207611, Optimal Rounds: 1
Combination 4/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 1, MSE: 0.20405831284954176, Optimal Rounds: 80
Combination 5/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 5, MSE: 0.2084961494724242, Optimal Rounds: 274
Combination 6/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 10, MSE: 0.23978626605614342, Optimal Rounds: 1
Combination 7/81 - lambda: 0, max_depth: 3, learning_rate: 0.2, min_child_weight: 1, MSE: 0.20986889378807533, Optimal Rounds: 26
Combination 8/81 - lambda: 0, max_depth: 3, learning_rate: 0.

In [ ]:
%%time


# Define the parameter grid
lambda_values = [0,1,2]
max_depth_values = [3, 6, 9]
learning_rate_values = [0.01, 0.1, 0.2]
min_child_weight_values = [1, 5, 10]

# Prepare to collect results
best_params = None
best_score = float("inf")  # Initialize with a large number
best_num_boost_round = None
combination_count = 0  # Counter for combinations

# Calculate the total number of combinations
total_combinations = len(lambda_values) * len(max_depth_values) * len(learning_rate_values) * len(min_child_weight_values)
print("Total combinations: ", total_combinations)
xgb_train_matrix = xgb.DMatrix(x_2_ds,label=y_2_ds)
# Loop over the hyperparameter values
for lambda_value in lambda_values:
    for max_depth in max_depth_values:
        for learning_rate in learning_rate_values:
            for min_child_weight in min_child_weight_values:
                combination_count += 1  # Increment combination count
                params = {
                    "objective": "binary:logistic",
                    "max_depth": max_depth,
                    "learning_rate": learning_rate,
                    "subsample": 0.8,  # Assuming a default subsample value
                    "gamma": 0,  # Assuming a default gamma value
                    "booster": 'dart',
                    "rate_drop": 0.1 ,
                    "min_child_weight": min_child_weight,
                    "lambda": lambda_value,  # Set the L2 regularization strength
                    "device": "cuda"
                }

                # Perform cross-validation with early stopping
                cv_results = xgb.cv(
                    params, xgb_train_matrix,
                    num_boost_round=20,
                    nfold=5,
                    metrics=["rmse"],
                    seed=42, early_stopping_rounds=50
                )

                # Compute average RMSE across folds
                avg_rmse = cv_results['test-rmse-mean'].min() * cv_results['test-rmse-mean'].min()
                optimal_rounds = cv_results.shape[0]
                if avg_rmse < best_score:
                    best_score = avg_rmse
                    best_params = params
                    best_num_boost_round = optimal_rounds

                # Print information about the current combination
                print(f"Combination {combination_count}/{total_combinations} - lambda: {lambda_value}, max_depth: {max_depth}, learning_rate: {learning_rate}, min_child_weight: {min_child_weight}, MSE: {avg_rmse}, Optimal Rounds: {optimal_rounds}")

print("Best Parameters:", best_params)
print("Optimal num_boost_round:", best_num_boost_round)
print("Best Score (MSE):", best_score)
print("Total Combinations Processed:", total_combinations)


Total combinations:  81
Combination 1/81 - lambda: 0, max_depth: 3, learning_rate: 0.01, min_child_weight: 1, MSE: 0.2363682215879338, Optimal Rounds: 20
Combination 2/81 - lambda: 0, max_depth: 3, learning_rate: 0.01, min_child_weight: 5, MSE: 0.2393648002448218, Optimal Rounds: 20
Combination 3/81 - lambda: 0, max_depth: 3, learning_rate: 0.01, min_child_weight: 10, MSE: 0.2399288423220287, Optimal Rounds: 20
Combination 4/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 1, MSE: 0.2214580830504279, Optimal Rounds: 20
Combination 5/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 5, MSE: 0.23566232962448985, Optimal Rounds: 20
Combination 6/81 - lambda: 0, max_depth: 3, learning_rate: 0.1, min_child_weight: 10, MSE: 0.2401522813496665, Optimal Rounds: 20
Combination 7/81 - lambda: 0, max_depth: 3, learning_rate: 0.2, min_child_weight: 1, MSE: 0.20941496388968803, Optimal Rounds: 20
Combination 8/81 - lambda: 0, max_depth: 3, learning_rate: 0.2, mi

CPU times: user 559 ms <br>
Sys: 3.23 ms <br>
Total: 562 ms <br>
Wall time: 313 ms

In [ ]:
%%time
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score

params = {'objective': 'binary:logistic', 'max_depth': 9, 'learning_rate': 0.2, 'subsample': 0.8, 'gamma': 0, 'booster': 'gbtree', 'min_child_weight': 1, 'lambda': 0, 'device': 'cuda'}

xgb_train_matrix = xgb.DMatrix(x_2_ds,label=y_2_ds)

model = xgb.train(params, xgb_train_matrix, num_boost_round=69, evals=[(xgb_train_matrix, "train")],
                  early_stopping_rounds=100, verbose_eval=False)

dtest = xgb.DMatrix(x_2_ts, label=y_2_ts)

# Predict labels for the training set and test set, converting probabilities to binary output
train_labels_pred = (model.predict(xgb_train_matrix) > 0.5).astype(int)
test_labels_pred = (model.predict(dtest) > 0.5).astype(int)

# Calculate accuracy for training and test datasets
train_accuracy = accuracy_score(y_2_ds, train_labels_pred)
test_accuracy = accuracy_score(y_2_ts, test_labels_pred)

# Output the results
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

train_mse = mean_squared_error(y_2_ds, train_labels_pred)
test_mse = mean_squared_error(y_2_ts, test_labels_pred)
print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")


Training Accuracy: 1.0
Test Accuracy: 0.7546296296296297
Training MSE: 0.0
Test MSE: 0.24537037037037038
CPU times: user 726 ms, sys: 80.9 ms, total: 806 ms
Wall time: 561 ms


# MONK 3

In [ ]:
df = pd.read_csv('MONK/monks-3.train', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])
df_2 = pd.read_csv('MONK/monks-3.test', delim_whitespace=True, low_memory=False, header = None, names = ['y',1,2,3,4,5,6], usecols=[0,1,2,3,4,5,6])


## Applicazione on hot encoding e split dataset

In [ ]:
monk3_tr=one_hot_encode(df, [1,2,3,4,5,6])
monk3_ts=one_hot_encode(df_2, [1,2,3,4,5,6])
training_3_x, validation_3_x, training_3_y, validation_3_y = train_test_split(monk3_tr, df[['y']], stratify =None, test_size=0.30, random_state = 42, shuffle=False)

x_3_ts = monk3_ts.iloc[:,1:].values #ottiene tutte le colonne con indice >=1, scartando la target label
y_3_ts = monk3_ts.iloc[:,0].values # ottiene solo colonne con indice 0, solo la target label


x_3_tr = training_3_x.iloc[:,1:].values
y_3_tr = training_3_y.iloc[:,:].values
x_3_vl = validation_3_x.iloc[:,1:].values
y_3_vl = validation_3_y.iloc[:,:].values

x_3_ds = monk3_tr.iloc[:,1:].values
y_3_ds = monk3_tr.iloc[:,0].values



## Search

In [ ]:
xgb_train_matrix = xgb.DMatrix(x_3_ds,label=y_3_ds)

valutiamo il modello rispetto al numero di round di boosting.

CPU times: user 48.2 s <br>
Sys: 6.15 s <br>
Total: 54.3 s <br>
Wall time: 54.7 s

In [ ]:
%%time


# Define the parameter grid
lambda_values = [1,2,3]
max_depth_values = [3, 6, 9]
learning_rate_values = [0.05, 0.1, 0.20]
min_child_weight_values = [1, 10, 100]

# Prepare to collect results
best_params = None
best_score = float("inf")  # Initialize with a large number
best_num_boost_round = None
combination_count = 0  # Counter for combinations

# Calculate the total number of combinations
total_combinations = len(lambda_values) * len(max_depth_values) * len(learning_rate_values) * len(min_child_weight_values)
print("Total combinations: ", total_combinations)

# Loop over the hyperparameter values
for lambda_value in lambda_values:
    for max_depth in max_depth_values:
        for learning_rate in learning_rate_values:
            for min_child_weight in min_child_weight_values:
                combination_count += 1  # Increment combination count
                params = {
                    "objective": "binary:logistic",
                    "max_depth": max_depth,
                    "learning_rate": learning_rate,
                    "subsample": 0.8,  # Assuming a default subsample value
                    "gamma": 0,  # Assuming a default gamma value
                    "booster": 'gbtree',
                    "min_child_weight": min_child_weight,
                    "lambda": lambda_value,  # Set the L2 regularization strength
                    "device": "cuda"
                }

                # Perform cross-validation with early stopping
                cv_results = xgb.cv(
                    params, xgb_train_matrix,
                    num_boost_round=1000,
                    nfold=5,
                    metrics=["rmse"],  # Assuming you're looking for RMSE as a regression metric
                    seed=42, early_stopping_rounds=10
                )

                # Compute average RMSE across folds
                avg_rmse = cv_results['test-rmse-mean'].min() * cv_results['test-rmse-mean'].min()
                optimal_rounds = cv_results.shape[0]
                if avg_rmse < best_score:
                    best_score = avg_rmse
                    best_params = params
                    best_num_boost_round = optimal_rounds  # Updated to optimal rounds

                # Print information about the current combination
                print(f"Combination {combination_count}/{total_combinations} - lambda: {lambda_value}, max_depth: {max_depth}, learning_rate: {learning_rate}, min_child_weight: {min_child_weight}, MSE: {avg_rmse}, Optimal Rounds: {optimal_rounds}")

print("Best Parameters:", best_params)
print("Optimal num_boost_round:", best_num_boost_round)
print("Best Score (MSE):", best_score)
print("Total Combinations Processed:", total_combinations)


Total combinations:  81
Combination 1/81 - lambda: 1, max_depth: 3, learning_rate: 0.05, min_child_weight: 1, MSE: 0.06022468305082498, Optimal Rounds: 131
Combination 2/81 - lambda: 1, max_depth: 3, learning_rate: 0.05, min_child_weight: 10, MSE: 0.2536333399426305, Optimal Rounds: 2
Combination 3/81 - lambda: 1, max_depth: 3, learning_rate: 0.05, min_child_weight: 100, MSE: 0.2535858194494735, Optimal Rounds: 1
Combination 4/81 - lambda: 1, max_depth: 3, learning_rate: 0.1, min_child_weight: 1, MSE: 0.061382944656341025, Optimal Rounds: 53
Combination 5/81 - lambda: 1, max_depth: 3, learning_rate: 0.1, min_child_weight: 10, MSE: 0.25368572388515714, Optimal Rounds: 2
Combination 6/81 - lambda: 1, max_depth: 3, learning_rate: 0.1, min_child_weight: 100, MSE: 0.2535858194494735, Optimal Rounds: 1
Combination 7/81 - lambda: 1, max_depth: 3, learning_rate: 0.2, min_child_weight: 1, MSE: 0.058862847783539875, Optimal Rounds: 25
Combination 8/81 - lambda: 1, max_depth: 3, learning_rate: 0.

CPU times: user 447 ms <br>
Sys: 18.5 ms <br>
Total: 465 ms <br>
Wall time: 237 ms

In [ ]:
%%time
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
xgb_train_matrix = xgb.DMatrix(x_3_ds,label=y_3_ds)
params = {'objective': 'binary:logistic', 'max_depth': 6, 'learning_rate': 0.2, 'subsample': 0.8, 'gamma': 0, 'booster': 'gbtree', 'min_child_weight': 1, 'lambda': 2, 'device': 'cuda'}

# Assuming 'dtrain' is already defined
model = xgb.train(params, xgb_train_matrix, num_boost_round=47, evals=[(xgb_train_matrix, "train")],
                  early_stopping_rounds=100, verbose_eval=False)

dtest = xgb.DMatrix(x_3_ts, label=y_3_ts)

# Predict labels for the training set and test set, converting probabilities to binary output
train_labels_pred = (model.predict(xgb_train_matrix) > 0.5).astype(int)
test_labels_pred = (model.predict(dtest) > 0.5).astype(int)

# Calculate accuracy for training and test datasets
train_accuracy = accuracy_score(y_3_ds, train_labels_pred)
test_accuracy = accuracy_score(y_3_ts, test_labels_pred)

# Output the results
print(f"Training Accuracy: {train_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

train_mse = mean_squared_error(y_3_ds, train_labels_pred)
test_mse = mean_squared_error(y_3_ts, test_labels_pred)
print(f"Training MSE: {train_mse}")
print(f"Test MSE: {test_mse}")


Training Accuracy: 0.9590163934426229
Test Accuracy: 0.9675925925925926
Training MSE: 0.040983606557377046
Test MSE: 0.032407407407407406
CPU times: user 447 ms, sys: 18.5 ms, total: 465 ms
Wall time: 237 ms
